In [1]:
def list_blobs(bucket_name, folder_name):
    """List all files in given COS directory."""    
    blob_names = []
    gcs_client = storage.Client()
    bucket = gcs_client.bucket(bucket_name)
    blobs = list(bucket.list_blobs(prefix=folder_name))
    for blob in blobs:
        blob_names.append(blob.name)
    return blob_names
    
def list_blobs_pd(bucket_name, folder_name):
    """List all files in given COS directory."""       
    gcs_client = storage.Client()
    bucket = gcs_client.bucket(bucket_name)
    blobs = list(bucket.list_blobs(prefix=folder_name))

    blob_name = []
    blob_size = []
    blob_time = []
    
    for blob in blobs:
        blob_name.append(blob.name)
        blob_size.append(blob.size)
        blob_time.append(blob.time_created)

    blobs_df = pd.DataFrame(list(zip(blob_name, blob_size, blob_time)), columns=['filePath', 'size', 'timeStamp'])    
    return blobs_df

def download_blob(bucket_name, source_blob_name, destination_file_name):
    """Downloads a blob from COS bucket."""
    gcs_client = storage.Client()
    bucket = gcs_client.bucket(bucket_name)
    blob = bucket.blob(source_blob_name)
    blob.download_to_filename(destination_file_name)

def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""    
    gcs_client = storage.Client()
    bucket = gcs_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_name)

In [8]:
import sqlite3
from google.cloud import storage
import pandas as pd

In [9]:
DB_NAME = "dev.db"
BUCKET_NAME = "data_tql"
FOLDER_NAME = "/WikiSQL/data"

In [10]:
blobs = list_blobs_pd(BUCKET_NAME, FOLDER_NAME)

In [11]:
list_blobs(BUCKET_NAME, FOLDER_NAME)

[]

In [12]:
conn = sqlite3.connect(DB_NAME)

In [13]:
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
schemas = cursor.fetchall()

IndexError: list index out of range

In [ ]:

conn = sqlite3.connect(DB_NAME)


# Loop through each schema and create a SQLite file for it
for schema in schemas:
    # Get the schema name and create a new SQLite file for it
    schema_name = schema[0]
    new_db_name = f"{schema_name}.sqlite"
    new_conn = sqlite3.connect(new_db_name)
    cursor = new_conn.cursor()

    # Export the schema to the new SQLite file
    cursor.execute(f"SELECT * INTO {schema_name} FROM {schema_name};")
    new_conn.commit()

    # Upload the SQLite file to GCP
    storage_client = storage.Client()
    bucket = storage_client.bucket(BUCKET_NAME)
    blob = bucket.blob(new_db_name)
    blob.upload_from_filename(new_db_name)

    # Close the connection to the new SQLite file
    new_conn.close()

# Close the connection to the .db file
conn.close()
